# Challenging Fake Image Detection from GAN Models

#### Detecting fake or manipulated images in today's digital age has become increasingly challenging due to the advancements in Generative Adversarial Networks (GANs). These AI-powered tools have made it easier than ever to create convincing fake images that can deceive both human observers and traditional image analysis techniques. The problem at hand is to develop a robust and effective fake image detection system that can differentiate between genuine and manipulated images generated by GAN models.

### By: Shashidhar Y Bhat

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

### Adjustable Parameters

In [2]:
train_dir = "./train/" 
test_dir = "./test/"
batch_size = 64
epochs = 20

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [4]:
transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(32),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
train_dataset = ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(128 * 4 * 4, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

model = CNN().to(device)

In [7]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Finished Training")

Epoch 1, Loss: 0.5101270869765157
Epoch 2, Loss: 0.42317306107805086
Epoch 3, Loss: 0.3851573493948977
Epoch 4, Loss: 0.3626243359983082
Epoch 5, Loss: 0.35065844432901894
Epoch 6, Loss: 0.3394696232567822
Epoch 7, Loss: 0.33204978202980295
Epoch 8, Loss: 0.32609326801884275
Epoch 9, Loss: 0.32009835934989816
Epoch 10, Loss: 0.31656102702660355
Epoch 11, Loss: 0.30983269454879175
Epoch 12, Loss: 0.3074976166356319
Epoch 13, Loss: 0.3031492276940678
Epoch 14, Loss: 0.30053549999715573
Epoch 15, Loss: 0.2973550752958882
Epoch 16, Loss: 0.29427219612699096
Epoch 17, Loss: 0.29256950057590153
Epoch 18, Loss: 0.28824524804520746
Epoch 19, Loss: 0.2894555256056694
Epoch 20, Loss: 0.2865192043384679
Finished Training


In [8]:
test_dataset = ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs >= 0.5).view(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on Test Dataset: {100 * correct / total:.2f}%")

Accuracy on Test Dataset: 87.90%
